# Inference Sample for ESM1nv

Copyright (c) 2022, NVIDIA CORPORATION. Licensed under the Apache License, Version 2.0 (the "License") you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.


### Prerequisite

- Linux OS
- Pascal, Volta, Turing, or an NVIDIA Ampere architecture-based GPU.
- NVIDIA Driver
- Docker



In [ ]:
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from pathlib import Path
import os

try:
    BIONEMO_HOME: Path = Path(os.environ['BIONEMO_HOME']).absolute()
except KeyError:
    print("Must have BIONEMO_HOME set in the environment! See docs for instructions.")
    raise

config_path = BIONEMO_HOME / "examples" / "protein" / "esm1nv" / "conf"
print(f"Using model configuration at: {config_path}")
assert config_path.is_dir()

### Setup and Test Data

In [ ]:
seqs = [
    'MSLKRKNIALIPAAGIGVRFGADKPKQYVEIGSKTVLEHVL', 
    'MIQSQINRNIRLDLADAILLSKAKKDLSFAEIADGTGLA',
]

In [ ]:
from bionemo.triton.utils import load_model_config

cfg = load_model_config(config_path, config_name="infer.yaml")

In [ ]:
from bionemo.triton.utils import load_model_for_inference
from bionemo.model.protein.esm1nv.infer import ESM1nvInference

inferer = load_model_for_inference(cfg, interactive=True)

print(f"Loaded a {type(inferer)}")
assert isinstance(inferer, ESM1nvInference)

### Sequence to Hidden States

`seq_to_hiddens` queries the model to fetch the encoder hidden states for the input protein sequence. `pad_masks` is returned with `hidden_states` and contains input padding information.

In [ ]:
hidden_states, pad_masks = inferer.seq_to_hiddens(seqs)
print(f"{hidden_states.shape=}")
print(f"{pad_masks.shape=}")
assert tuple(hidden_states.shape) == (2, 43, 768)
assert tuple(pad_masks.shape) == (2, 43)

### Hidden States to Embedding

__`hiddens_to_embedding`__ computes embedding vector by averaging `hidden_states` 

In [ ]:
embeddings = inferer.hiddens_to_embedding(hidden_states, pad_masks)
print(f"{embeddings.shape=}")
assert tuple(embeddings.shape) == (2, 768)

### Sequence to Embedding

__`seq_to_embedding`__  queries the model to fetch the encoder hidden states and computes embedding vector

In [ ]:
embeddings = inferer.seq_to_embeddings(seqs)
print(f"{embeddings.shape=}")
assert tuple(embeddings.shape) == (2, 768)